# Estimation for Rigid Transformation
+ Formalization
$$
q_i = R_{xyz} p_i + t + \epsilon, \epsilon \sim N(0,1),
$$
+ $t$ and $R_{xyz}$ is targets to estimate and $R_{xyz} = R_x(\theta_x) R_y(\theta_y) R_z(\theta_z)$.
+ Optimizing each parameters $t$, $R_x(\theta_x)$, $R_y(\theta_y)$, and $R_z(\theta_z)$ are considered here.
+ First, this notebook consider about $t$ and $R_x(\theta_x)$

In [2]:
import numpy as np

In [3]:
def rot_x(theta_x: float) -> np.ndarray:
    ct = np.cos(theta_x)
    st = np.sin(theta_x)
    return np.array([
        [1, 0, 0],
        [0, ct, -st],
        [0, st, ct]
    ])
    pass

def rot_y(theta_y: float) -> np.ndarray:
    ct = np.cos(theta_y)
    st = np.sin(theta_y)
    return np.array([
        [ct, 0, -st],
        [0, 1, 0],
        [st, 0, ct]
    ])

def rot_z(theta_z: float) -> np.ndarray:
    ct = np.cos(theta_z)
    st = np.sin(theta_z)
    return np.array([
        [ct, -st, 0],
        [st, ct, 0],
        [0, 0, 1],
    ])

## Data generation

In [3]:
n_sample = 50
data_seed = 20240630

In [4]:
np.random.seed(data_seed)

true_theta_x = np.random.uniform(low=0, high=2*np.pi)
true_trans_vec = np.random.normal(size=3)
true_rot = rot_x(true_theta_x)

p_i = np.random.normal(size=(n_sample, 3))
q_i = p_i @ true_rot.T + true_trans_vec + np.random.normal(scale=1, size=(n_sample, 3))

### log likelihood without constant term

In [5]:
-((q_i - (p_i @ true_rot.T + true_trans_vec))**2).sum(axis=1).sum()/2

-75.2004109842694

In [6]:
ln_seed = 20240701
np.random.seed(ln_seed)

### Initialize theta

In [7]:
est_theta_x = np.random.normal()

### estimated trans vec
+ translation vector is estimated simply by average of residuals

In [8]:
est_trans_vec = (q_i - p_i @ rot_x(est_theta_x).T).mean(axis=0)

### estimated rot mat
+ optimized function is
$$
L(\theta_x) = \sum_i <R_x(\theta_x)p_i, q_i - t>,
$$
where redundant terms to optimize $\theta_x$ are omitted.
By calculating max value of $L(\theta_x)$, estimated $\theta_x$ is obtained.

In [9]:
tilde_q_i = q_i - est_trans_vec
# tilde_q_i = q_i - true_trans_vec
alpha_x = (-p_i[:,2]*tilde_q_i[:,1] + p_i[:,1]*tilde_q_i[:,2]).sum()
beta_x = -(p_i[:,1]*tilde_q_i[:,1] + p_i[:,2]*tilde_q_i[:,2]).sum()

In [10]:
if beta_x >= 0:
    gamma_x = np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))
else:
    gamma_x = np.pi - np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))

In [11]:
candidate_est_theta_x = np.arange(4) * np.pi - gamma_x
candidate_est_theta_x = candidate_est_theta_x[(0<=candidate_est_theta_x) & (candidate_est_theta_x <= 2*np.pi)]
est_theta_x = min(candidate_est_theta_x[np.cos(candidate_est_theta_x + gamma_x) < 0])
# candidate_est_theta_x[np.argmax(alpha_x*np.sin(candidate_est_theta_x) + beta_x*np.cos(candidate_est_theta_x))]

## estimated by loop 

In [14]:
def est_theta(sin_coef: float, cos_coef: float) -> float:
    """ estimate maximizing theta of sin_coef sin(theta) + cos_coef cos(theta),
    where theta in [0,2pi].
    consider the 1st and 2nd order derivative, the value is obtained.
    """
    
    dist = np.sqrt(sin_coef**2 + cos_coef**2)
    if sin_coef >= 0:
        delta = np.arcsin(cos_coef/dist)
        pass
    else:
        delta = np.pi - np.arcsin(cos_coef/dist)
        pass
    candidate_est_theta = np.arange(4) * np.pi - delta
    est_theta = min(
        filter(
            lambda theta: 
            0 <= theta and theta <= 2*np.pi # for range of theta
            and np.cos(theta + delta) < 0, # for 2nd order derivative is negative
            candidate_est_theta,
        )
    )
    return est_theta
    pass

In [15]:
n_ite = 100
ln_seed = 20240701

In [16]:
loglik = []
np.random.seed(ln_seed)

# initialize
# est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_x
    tilde_q_i = q_i - est_trans_vec
    alpha_x = (-p_i[:,2]*tilde_q_i[:,1] + p_i[:,1]*tilde_q_i[:,2]).sum()
    beta_x = -(p_i[:,1]*tilde_q_i[:,1] + p_i[:,2]*tilde_q_i[:,2]).sum()
    est_theta_x = est_theta(sin_coef=beta_x, cos_coef=alpha_x)

    # estimate translation vector
    est_trans_vec = (q_i - p_i @ rot_x(est_theta_x).T).mean(axis=0)
    
    est_rot = rot_x(est_theta_x)
    loglik.append(-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)

In [17]:
loglik[:10]

[-74.3712514179364,
 -74.24325155589148,
 -74.2432501645077,
 -74.24325016449257,
 -74.24325016449255,
 -74.24325016449257,
 -74.24325016449257,
 -74.24325016449257,
 -74.24325016449257,
 -74.24325016449257]

In [18]:
est_theta_x, est_trans_vec

(4.186908785248823, array([ 0.33507599, -0.24968013, -0.99294305]))

In [19]:
true_theta_x, true_trans_vec

(4.247931759760334, array([ 0.34174339, -0.35949451, -0.85681546]))

## Optimize $\theta_x$ and $\theta_y$ and $t$

In [20]:
n_sample = 50
data_seed = 20240630

In [21]:
np.random.seed(data_seed)

true_theta_x = np.random.uniform(low=0, high=2*np.pi)
true_theta_y = np.random.uniform(low=0, high=2*np.pi)
true_trans_vec = np.random.normal(size=3)
true_rot = rot_x(true_theta_x) @ rot_y(true_theta_y)

p_i = np.random.normal(size=(n_sample, 3))
q_i = p_i @ true_rot.T + true_trans_vec + np.random.normal(scale=1, size=(n_sample, 3))

### log likelihood without constant term

In [22]:
-((q_i - (p_i @ true_rot.T + true_trans_vec))**2).sum(axis=1).sum()/2

-90.0363616229746

### Initialize translation vector and $\theta_x$

In [23]:
ln_seed = 20240701
np.random.seed(ln_seed)

In [24]:
est_trans_vec = np.random.normal(size=3)
est_theta_x = np.random.uniform(low=0, high=2*np.pi)

### Estimate $\theta_y$

In [ ]:
tilde_q_i = (q_i - est_trans_vec) @ rot_x(est_theta_x)

sin_coef = -(p_i[:,0]*tilde_q_i[:,0] + p_i[:,2]*tilde_q_i[:,2]).sum()
cos_coef = (-p_i[:,2]*tilde_q_i[:,0] + p_i[:,0]*tilde_q_i[:,2]).sum()

est_theta_y = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)

## Estimate $\theta_x$

In [ ]:
tilde_q_i = q_i - est_trans_vec
tilde_p_i = p_i @ rot_y(est_theta_y).T

sin_coef = -(tilde_p_i[:,1]*tilde_q_i[:,1] + tilde_p_i[:,2]*tilde_q_i[:,2]).sum()
cos_coef = (-tilde_p_i[:,2]*tilde_q_i[:,1] + tilde_p_i[:,1]*tilde_q_i[:,2]).sum()

est_theta_x = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)

## Esimate $t$

In [32]:
est_trans_vec = (q_i - p_i @ rot_y(est_theta_y).T @ rot_x(est_theta_x).T).mean(axis=0)

In [33]:
est_theta_x, est_theta_y

(4.134738317936312, 1.1139391311678266)

In [34]:
est_trans_vec

array([-1.51060822,  1.47331913,  1.14509845])

In [35]:
true_theta_x, true_theta_y, true_trans_vec

(4.247931759760334,
 1.0004734530637451,
 array([-1.33375912,  1.30214425,  1.31281983]))

# Loop to estimate R_x, R_y, R_z, and t

In [68]:
n_sample = 50
data_seed = 20240630

In [69]:
np.random.seed(data_seed)

true_theta_x = np.random.uniform(low=0, high=2*np.pi)
true_theta_y = np.random.uniform(low=0, high=2*np.pi)
true_theta_z = np.random.uniform(low=0, high=2*np.pi)
true_trans_vec = np.random.normal(size=3)
true_rot = rot_x(true_theta_x) @ rot_y(true_theta_y) @ rot_z(true_theta_z)

p_i = np.random.normal(size=(n_sample, 3))
q_i = p_i @ true_rot.T + true_trans_vec + np.random.normal(scale=1, size=(n_sample, 3))

In [74]:
def est_theta(sin_coef: float, cos_coef: float) -> float:
    """ estimate maximizing theta of sin_coef sin(theta) + cos_coef cos(theta),
    where theta in [0,2pi].
    consider the 1st and 2nd order derivative, the value is obtained.
    """
    
    dist = np.sqrt(sin_coef**2 + cos_coef**2)
    if sin_coef >= 0:
        delta = np.arcsin(cos_coef/dist)
        pass
    else:
        delta = np.pi - np.arcsin(cos_coef/dist)
        pass
    candidate_est_theta = np.arange(4) * np.pi - delta
    est_theta = min(
        filter(
            lambda theta: 
            0 <= theta and theta <= 2*np.pi # for range of theta
            and np.cos(theta + delta) < 0, # for 2nd order derivative is negative
            candidate_est_theta,
        )
    )
    return est_theta
    pass

In [78]:
n_ite = 100
ln_seed = 20240701

In [79]:
loglik = []
np.random.seed(ln_seed)

# initialize
est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_theta_y = np.random.uniform(low=0, high=2*np.pi)
est_theta_z = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_z
    tilde_q_i = (q_i - est_trans_vec) @ rot_x(est_theta_x) @ rot_y(est_theta_y)
    tilde_p_i = p_i
    sin_coef = -(tilde_p_i[:,0]*tilde_q_i[:,0] + tilde_p_i[:,1]*tilde_q_i[:,1]).sum()
    cos_coef = (-tilde_p_i[:,1]*tilde_q_i[:,0] + tilde_p_i[:,0]*tilde_q_i[:,1]).sum()    
    est_theta_z = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)
    
    # estimate theta_y
    tilde_q_i = (q_i - est_trans_vec) @ rot_x(est_theta_x)
    tilde_p_i = p_i @ rot_z(est_theta_z).T
    sin_coef = -(tilde_p_i[:,0]*tilde_q_i[:,0] + tilde_p_i[:,2]*tilde_q_i[:,2]).sum()
    cos_coef = (-tilde_p_i[:,2]*tilde_q_i[:,0] + tilde_p_i[:,0]*tilde_q_i[:,2]).sum()
    est_theta_y = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)
    
    # estimate theta_x
    tilde_q_i = q_i - est_trans_vec
    tilde_p_i = p_i @ rot_z(est_theta_z).T @ rot_y(est_theta_y).T    
    sin_coef = -(tilde_p_i[:,1]*tilde_q_i[:,1] + tilde_p_i[:,2]*tilde_q_i[:,2]).sum()
    cos_coef = (-tilde_p_i[:,2]*tilde_q_i[:,1] + tilde_p_i[:,1]*tilde_q_i[:,2]).sum()
    est_theta_x = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)

    # estimate translation vector
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    est_trans_vec = (q_i - p_i @ est_rot.T).mean(axis=0)
    
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    loglik.append(-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)

In [80]:
loglik

[-75.35936256579465,
 -73.80151013457045,
 -73.61108814470299,
 -73.54158777442599,
 -73.51571188186384,
 -73.50610635006676,
 -73.50254007113755,
 -73.5012146496122,
 -73.50072157272523,
 -73.5005380084679,
 -73.50046963730414,
 -73.50044416346446,
 -73.50043467049603,
 -73.50043113245127,
 -73.50042981371558,
 -73.50042932216006,
 -73.50042913892867,
 -73.50042907062644,
 -73.5004290451655,
 -73.50042903567437,
 -73.50042903213634,
 -73.50042903081749,
 -73.50042903032582,
 -73.50042903014254,
 -73.50042903007422,
 -73.50042903004875,
 -73.50042903003926,
 -73.50042903003572,
 -73.50042903003441,
 -73.50042903003391,
 -73.50042903003374,
 -73.50042903003364,
 -73.50042903003364,
 -73.50042903003363,
 -73.50042903003364,
 -73.50042903003364,
 -73.50042903003363,
 -73.5004290300336,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003362,
 -73.50042903003363,
 -73.50042903003364,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003362,
 -73.500429030033

# Loop to estimate R_x, R_y, R_z, and t with a covariance noise

In [88]:
import scipy

In [1]:
cov = scipy.stats.wishart.rvs(df=5, scale=np.eye(3))

NameError: name 'scipy' is not defined

In [ ]:
cov

In [87]:
np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=1)

array([[ 0.10763867,  0.73118401, -0.25001827]])

In [68]:
n_sample = 50
data_seed = 20240630

In [69]:
np.random.seed(data_seed)

true_theta_x = np.random.uniform(low=0, high=2*np.pi)
true_theta_y = np.random.uniform(low=0, high=2*np.pi)
true_theta_z = np.random.uniform(low=0, high=2*np.pi)
true_trans_vec = np.random.normal(size=3)
true_rot = rot_x(true_theta_x) @ rot_y(true_theta_y) @ rot_z(true_theta_z)

p_i = np.random.normal(size=(n_sample, 3))
q_i = p_i @ true_rot.T + true_trans_vec + np.random.normal(scale=1, size=(n_sample, 3))

In [74]:
def est_theta(sin_coef: float, cos_coef: float) -> float:
    """ estimate maximizing theta of sin_coef sin(theta) + cos_coef cos(theta),
    where theta in [0,2pi].
    consider the 1st and 2nd order derivative, the value is obtained.
    """
    
    dist = np.sqrt(sin_coef**2 + cos_coef**2)
    if sin_coef >= 0:
        delta = np.arcsin(cos_coef/dist)
        pass
    else:
        delta = np.pi - np.arcsin(cos_coef/dist)
        pass
    candidate_est_theta = np.arange(4) * np.pi - delta
    est_theta = min(
        filter(
            lambda theta: 
            0 <= theta and theta <= 2*np.pi # for range of theta
            and np.cos(theta + delta) < 0, # for 2nd order derivative is negative
            candidate_est_theta,
        )
    )
    return est_theta
    pass

In [78]:
n_ite = 100
ln_seed = 20240701

In [79]:
loglik = []
np.random.seed(ln_seed)

# initialize
est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_theta_y = np.random.uniform(low=0, high=2*np.pi)
est_theta_z = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_z
    tilde_q_i = (q_i - est_trans_vec) @ rot_x(est_theta_x) @ rot_y(est_theta_y)
    tilde_p_i = p_i
    sin_coef = -(tilde_p_i[:,0]*tilde_q_i[:,0] + tilde_p_i[:,1]*tilde_q_i[:,1]).sum()
    cos_coef = (-tilde_p_i[:,1]*tilde_q_i[:,0] + tilde_p_i[:,0]*tilde_q_i[:,1]).sum()    
    est_theta_z = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)
    
    # estimate theta_y
    tilde_q_i = (q_i - est_trans_vec) @ rot_x(est_theta_x)
    tilde_p_i = p_i @ rot_z(est_theta_z).T
    sin_coef = -(tilde_p_i[:,0]*tilde_q_i[:,0] + tilde_p_i[:,2]*tilde_q_i[:,2]).sum()
    cos_coef = (-tilde_p_i[:,2]*tilde_q_i[:,0] + tilde_p_i[:,0]*tilde_q_i[:,2]).sum()
    est_theta_y = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)
    
    # estimate theta_x
    tilde_q_i = q_i - est_trans_vec
    tilde_p_i = p_i @ rot_z(est_theta_z).T @ rot_y(est_theta_y).T    
    sin_coef = -(tilde_p_i[:,1]*tilde_q_i[:,1] + tilde_p_i[:,2]*tilde_q_i[:,2]).sum()
    cos_coef = (-tilde_p_i[:,2]*tilde_q_i[:,1] + tilde_p_i[:,1]*tilde_q_i[:,2]).sum()
    est_theta_x = est_theta(sin_coef=sin_coef, cos_coef=cos_coef)

    # estimate translation vector
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    est_trans_vec = (q_i - p_i @ est_rot.T).mean(axis=0)
    
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    loglik.append(-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)

In [80]:
loglik

[-75.35936256579465,
 -73.80151013457045,
 -73.61108814470299,
 -73.54158777442599,
 -73.51571188186384,
 -73.50610635006676,
 -73.50254007113755,
 -73.5012146496122,
 -73.50072157272523,
 -73.5005380084679,
 -73.50046963730414,
 -73.50044416346446,
 -73.50043467049603,
 -73.50043113245127,
 -73.50042981371558,
 -73.50042932216006,
 -73.50042913892867,
 -73.50042907062644,
 -73.5004290451655,
 -73.50042903567437,
 -73.50042903213634,
 -73.50042903081749,
 -73.50042903032582,
 -73.50042903014254,
 -73.50042903007422,
 -73.50042903004875,
 -73.50042903003926,
 -73.50042903003572,
 -73.50042903003441,
 -73.50042903003391,
 -73.50042903003374,
 -73.50042903003364,
 -73.50042903003364,
 -73.50042903003363,
 -73.50042903003364,
 -73.50042903003364,
 -73.50042903003363,
 -73.5004290300336,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003362,
 -73.50042903003363,
 -73.50042903003364,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003363,
 -73.50042903003362,
 -73.500429030033

### estimated rot mat
+ optimized function is
$$
L(\theta_x) = \sum_i <R_x(\theta_x)p_i, q_i - t>,
$$
where redundant terms to optimize $\theta_x$ are omitted.
By calculating max value of $L(\theta_x)$, estimated $\theta_x$ is obtained.

In [36]:
tilde_q_i = q_i - est_trans_vec
# tilde_q_i = q_i - true_trans_vec
alpha_x = (-p_i[:,2]*tilde_q_i[:,1] + p_i[:,1]*tilde_q_i[:,2]).sum()
beta_x = -(p_i[:,1]*tilde_q_i[:,1] + p_i[:,2]*tilde_q_i[:,2]).sum()

In [37]:
if beta_x >= 0:
    gamma_x = np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))
else:
    gamma_x = np.pi - np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))

In [38]:
candidate_est_theta_x = np.arange(4) * np.pi - gamma_x
candidate_est_theta_x = candidate_est_theta_x[(0<=candidate_est_theta_x) & (candidate_est_theta_x <= 2*np.pi)]
est_theta_x = min(candidate_est_theta_x[np.cos(candidate_est_theta_x + gamma_x) < 0])
# candidate_est_theta_x[np.argmax(alpha_x*np.sin(candidate_est_theta_x) + beta_x*np.cos(candidate_est_theta_x))]

In [39]:
est_rot = rot_x(est_theta_x)
-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2

-163.38790035034546

In [40]:
true_theta_x

4.247931759760334

In [41]:
est_theta_x

4.262658203214274

In [42]:
est_theta_x

4.262658203214274

In [43]:
[np.cos(candidate_est_theta_x + gamma_x) < 0]

[array([False,  True])]

In [44]:
est_trans_vec, np.rad2deg(est_theta_x), np.rad2deg(true_theta_x)

(array([-1.51060822,  1.47331913,  1.14509845]),
 244.2323245509967,
 243.3885614938479)

In [45]:
rot_x(true_theta_x)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.44793759,  0.89406483],
       [ 0.        , -0.89406483, -0.44793759]])

In [46]:
rot_x(est_theta_x)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.4347231 ,  0.90056417],
       [ 0.        , -0.90056417, -0.4347231 ]])

In [47]:
est_theta_x

4.262658203214274

In [48]:
np.rad2deg(est_theta_x)

244.2323245509967

In [49]:
np.rad2deg(true_theta_x)

243.3885614938479

In [50]:
est_theta_x

4.262658203214274

In [51]:
est_trans_vec

array([-1.51060822,  1.47331913,  1.14509845])

In [52]:
true_trans_vec

array([-1.33375912,  1.30214425,  1.31281983])

In [53]:
np.sqrt(alpha_x**2 + beta_x**2) * np.sin(est_theta_x + gamma_x)

5.037827037954152e-14

In [54]:
alpha_x * np.cos(est_theta_x) - beta_x * np.sin(est_theta_x)

69.62406943783711

In [55]:
gamma_x = np.arccos(-beta_x / np.sqrt(alpha_x**2 + beta_x**2))
candidate_est_theta_x = np.arange(3) * np.pi - gamma_x
est_theta_x = min(candidate_est_theta_x[(0<=candidate_est_theta_x) & (candidate_est_theta_x <= 2*np.pi)].tolist())

In [56]:
np.rad2deg(true_theta_x)

243.3885614938479

In [57]:
np.rad2deg(est_theta_x)

64.23232455099672